In [2]:
from datasets import load_dataset, Dataset, DatasetDict, Audio

In [37]:
RECORDINGS_DIR="/run/media/johnny/31c5407a-2da6-4ef8-95ec-d294c1afec38/" 
META_FILE="../malorca/metadata_dev12.json"
SAVE_DIR="malorca"

# load the metadata
# meta_data = json.load(open(META_FILE, mode='r'))
dataset = load_dataset("json", data_files=META_FILE, split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [38]:
# set properly path to the recordings according to the current disk path
dataset = dataset.map(lambda x: {"audio": RECORDINGS_DIR + x["audio"]}, remove_columns=["audio"])

Map:   0%|          | 0/6335 [00:00<?, ? examples/s]

In [39]:
dataset[0]

{'audio': '/run/media/johnny/31c5407a-2da6-4ef8-95ec-d294c1afec38/MALORCA/DATA_ATC/VIENNA/WAV_FILES/LOWW07/BALAD_20160701_B7_1/2016-07-01__05-30-12-01.wav',
 'full_ts': 'austrian four five five november continue on present heading climb flight level two four zero',
 'short_ts': 'austrian 455N continue on present heading climb flight level 240 \n'}

In [40]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [41]:
dataset.save_to_disk(SAVE_DIR)

Saving the dataset (0/1 shards):   0%|          | 0/6335 [00:00<?, ? examples/s]

In [31]:
dd = Dataset.load_from_disk(SAVE_DIR)

In [32]:
dd

Dataset({
    features: ['audio', 'full_ts', 'short_ts'],
    num_rows: 3987
})

In [33]:
dd[0]

{'audio': {'path': '2016-07-01__05-30-12-01.wav',
  'array': array([-5.45940537e-04, -2.77002080e-04,  5.77722967e-04, ...,
         -1.16415322e-10,  2.91038305e-11,  2.91038305e-11]),
  'sampling_rate': 16000},
 'full_ts': 'austrian four five five november continue on present heading climb flight level two four zero',
 'short_ts': 'austrian 455N continue on present heading climb flight level 240 \n'}